In [ ]:
!pip install ultralytics


In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
vname = "test2.0Ch12 20240311121534.mp4"


In [ ]:
!yolo track source=vname save=True

track

In [ ]:
from collections import defaultdict

import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

# Dictionary to store tracking history with default empty lists
track_history = defaultdict(lambda: [])

# Load the YOLO model with segmentation capabilities
model = YOLO("yolov8n-seg.pt")

# Open the video file
cap = cv2.VideoCapture("Ch12 20240312083049.mp4")

# Retrieve video properties: width, height, and frames per second
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Initialize video writer to save the output video with the specified properties
out = cv2.VideoWriter("instance-segmentation-object-tracking.avi", cv2.VideoWriter_fourcc(*"MJPG"), fps, (w, h))

while True:
    # Read a frame from the video
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Create an annotator object to draw on the frame
    annotator = Annotator(im0, line_width=2)

    # Perform object tracking on the current frame
    #results = model.track(im0, persist=True)
    results = model(source=im0, conf=0.4)
    

    # Check if tracking IDs and masks are present in the results
    if results[0].boxes.id is not None and results[0].masks is not None:
        # Extract masks and tracking IDs
        masks = results[0].masks.xy
        track_ids = results[0].boxes.id.int().c().tolist()

        # Annotate each mask with its corresponding tracking ID and color
        for mask, track_id in zip(masks, track_ids):
            annotator.seg_bbox(mask=mask, mask_color=colors(track_id, True), track_label=str(track_id))

    # Write the annotated frame to the output video
    out.write(im0)
    # Display the annotated frame
    cv2.imshow("instance-segmentation-object-tracking", im0)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video writer and capture objects, and close all OpenCV windows
out.release()
cap.release()
cv2.destroyAllWindows()

In [ ]:
torch.zeros(1).cuda()

In [ ]:
from ultralytics import YOLO
import cv2
import torch
torch.cuda.set_device(0)


In [ ]:
vname = "test1.2Ch18 20240311101732.mp4"


In [ ]:

model = YOLO("yolov8l.pt" )

#results = model(source="test1Ch18 20240311101732.mp4", show=True ,  save = True)

cap = cv2.VideoCapture(vname)
frames_tracked = []

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        #cv2.imshow("YOLOv8 Inference", annotated_frame)
        send_video_frame(annotated_frame)
        
        #frames_tracked.append(annotated_frame)
        
        # Break the loop if 'q' is pressed
        #if cv2.waitKey(1) & 0xFF == ord("q"):
            #break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

sendin data to server

In [1]:
import socketio
import cv2
import base64
import numpy as np

# Create a Socket.IO client
sio = socketio.Client()

@sio.event
def connect():
    print('Connection established')

@sio.event
def disconnect():
    print('Disconnected from server')

@sio.event
def video_frame(data):
    print('Received video frame')
    try:
        # Debugging: Print the size of the data received
        print(f'Size of received data: {len(data)}')
        
        # Deserialize the received frame
        np_data = np.frombuffer(base64.b64decode(data), np.uint8)
        
        # Debugging: Check the shape of np_data before decoding
        print(f'Shape of np_data: {np_data.shape}')
        
        frame = cv2.imdecode(np_data, cv2.IMREAD_COLOR)
        
        if frame is None:
            print('Error: Frame is None after imdecode')
        else:
            cv2.imshow('Received Video Frame', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            sio.disconnect()
    except Exception as e:
        print(f'Error in video_frame handler: {e}')

def send_video_frame(frame):
    try:
        # Serialize the frame to send it over the socket
        _, buffer = cv2.imencode('.jpg', frame)
        
        # Debugging: Print the size of the buffer
        print(f'Size of buffer: {buffer.size}')
        
        frame_data = base64.b64encode(buffer).decode('utf-8')
        sio.emit('video_frame', frame_data)
    except Exception as e:
        print(f'Error in send_video_frame: {e}')

# Connect to the server
sio.connect('http://localhost:3000')

# Capture video from webcam and send frames to the server
'''
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    send_video_frame(frame)
'''
'''
cap.release()
cv2.destroyAllWindows()
sio.disconnect()
'''

Connection established


'\ncap.release()\ncv2.destroyAllWindows()\nsio.disconnect()\n'

In [ ]:
from PIL import Image
import numpy as np


In [ ]:
dim = Image.fromarray(cv2.cvtColor(frames_tracked[0], cv2.COLOR_BGR2RGB)).size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter('YOLO2tracked'+vname, fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(frame)
video_tracked.release()